In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel
from typing import Dict

In [2]:
load_dotenv(override=True)

True

In [6]:
openai_api_key = os.getenv('OPENAI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
llma_api_key = os.getenv('LLAMA_API_KEY')

In [7]:
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
LLMA_BASE_URL = "http://localhost:11434/v1"

In [9]:
deepseek_model_name = 'deepseek-chat'
llma_model_name = 'llama3.2'

In [8]:
deepseek_client = AsyncOpenAI(
    base_url = DEEPSEEK_BASE_URL, 
    api_key = deepseek_api_key
)

llma_client = AsyncOpenAI(
    base_url = LLMA_BASE_URL,
    api_key = llma_api_key
)

In [13]:
deepseek_model = OpenAIChatCompletionsModel(model=deepseek_model_name, openai_client=deepseek_client)
llma_model = OpenAIChatCompletionsModel(model=llma_model_name, openai_client=llma_client)

In [14]:
instructions1 = "You are witty sales agent for PKAI. You write professional and mindful emails"
instructions2 = "You are humorous sales agent for PKAI. You write professional and likely to get response emails"
instructions3 = "You are busy sales agent for PKAI. You write professional and concise emails"
agent1 = Agent(name="Agent 1", instructions=instructions1, model="gpt-4o-mini")
agent2 = Agent(name="Agent 2", instructions=instructions2, model=deepseek_model)
agent3 = Agent(name="Agent 3", instructions=instructions3, model=llma_model)

In [ ]:
agent1_tool = agent1.as_tool(tool_name='agent_1_tool', tool_description='Write a cold email')
agent2_tool = agent2.as_tool(tool_name='agent_2_tool', tool_description='Write a cold email')
agent3_tool = agent3.as_tool(tool_name='agent_3_tool', tool_description='Write a cold email')

In [16]:
@function_tool
def send_email(subject:str, html_body:str) -> Dict[str, str]:
    sg_client = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("hello@priyanshukhandelwal.com")
    to_email = To("udemy@priyanshukhandelwal.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg_client.send(mail)
    print(f"Email sent with status code: {response.status_code}")
    return response

NameError: name 'Dict' is not defined